In [2]:
import requests
import pandas as pd

# URL de la API
url = "https://apps.sec.cl/INTONLINEv1/ClientesAfectados/GetPorFecha"

# Headers (usando los más relevantes)
headers = {
    "Content-Type": "application/json; charset=UTF-8",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Origin": "https://apps.sec.cl",
    "Referer": "https://apps.sec.cl/INTONLINEv1/index.aspx",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

# Payload con la fecha que quieres consultar (ajusta según sea necesario)
payload = {
    "anio": 2025,
    "mes": 1,
    "dia": 29,
    "hora": 12
}

# Hacer la solicitud POST
response = requests.post(url, headers=headers, json=payload)

# Verificar si la respuesta es exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta a JSON

    # Convertir los datos en un DataFrame de pandas
    df = pd.DataFrame(data)

    # Guardar en un archivo CSV
    df.to_csv("clientes_afectados.csv", index=False, encoding="utf-8-sig")

    # Guardar en un archivo Excel
    df.to_excel("clientes_afectados.xlsx", index=False)

    print("✅ Datos guardados exitosamente en 'clientes_afectados.csv' y 'clientes_afectados.xlsx'")
else:
    print("❌ Error en la solicitud:", response.status_code, response.text)


✅ Datos guardados exitosamente en 'clientes_afectados.csv' y 'clientes_afectados.xlsx'


[<script type="text/javascript">
 /* <![CDATA[ */
 window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/15.0.3\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/15.0.3\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\/\/www.sec.cl\/sitio-web\/wp-includes\/js\/wp-emoji-release.min.js?ver=74300fa0682acf6d2f55fabd138da0a1"}};
 /*! This file is auto-generated */
 !function(i,n){var o,s,e;function c(e){try{var t={supportTests:e,timestamp:(new Date).valueOf()};sessionStorage.setItem(o,JSON.stringify(t))}catch(e){}}function p(e,t,n){e.clearRect(0,0,e.canvas.width,e.canvas.height),e.fillText(t,0,0);var t=new Uint32Array(e.getImageData(0,0,e.canvas.width,e.canvas.height).data),r=(e.clearRect(0,0,e.canvas.width,e.canvas.height),e.fillText(n,0,0),new Uint32Array(e.getImageData(0,0,e.canvas.width,e.canvas.height).data));return t.every(function(e,t){return e===r[t]})}function u(e,t,n){switch(t){case"flag":return n(e,"\ud83c\udff3\ufe0f\u200d\u2